In [1]:
import numpy as np
import cv2
import cvzone
import pickle

In [2]:
def toshow(name, img):
    width = 1100
    height = 650
    cv2.namedWindow(name, cv2.WINDOW_KEEPRATIO)
    cv2.resizeWindow(name, width, height)
    cv2.imshow(name, img)

In [3]:
width, height = 47, 17

try:
    with open("CarPos", "rb") as f:
        posList = pickle.load(f)
except:
    posList = []
    

def mouseClick(events,x,y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:
        posList.append((x,y))
    if events == cv2.EVENT_RBUTTONDOWN:
        for i, pos in enumerate(posList):
            x1, y1 = pos
            if x1<x<x1+width and y1<y<y1+height:
                posList.pop(i)
    
    with open('CarPos', 'wb') as f:
        pickle.dump(posList, f)


while True:
    img = cv2.imread("CarParkBigImg.jpg")
    for pos in posList:
        cv2.rectangle(img, pos, (pos[0]+width,pos[1]+height), (0,255,0), 2)
    
    toshow("Image", img)
    
    cv2.setMouseCallback("Image", mouseClick)
    
    k = cv2.waitKey(1)
    if k==ord("q"):
        cv2.destroyAllWindows()
        break

In [4]:
def CheckParking(imgPro):
    freePark =0
    for pos in posList:
        x, y = pos
        imgCrop = imgPro[y:y+height, x:x+width]
#     cv2.imshow("I", imgCrop)
        count = cv2.countNonZero(imgCrop)
        cvzone.putTextRect(img, str(count), (x+2, y+height-5), scale=1, thickness=2, offset=0, colorR=(255,0,255))
        if count<101:
            thickness = 2
            color=(0,255,0)
            freePark+=1
        else:
            thickness = 1
            color=(0,0,255)
        cv2.rectangle(img, pos, (pos[0]+width, pos[1]+height), color, thickness)
        cvzone.putTextRect(img, f'Free: {freePark}/{len(posList)}',(20,50), colorR = (0,255,0), offset=10, thickness=3) 

In [ ]:
vid = cv2.VideoCapture("CarParkResized.mp4")

with open("CarPos", "rb") as f:
    posList = pickle.load(f)

while True:
    if vid.get(cv2.CAP_PROP_POS_FRAMES) == vid.get(cv2.CAP_PROP_FRAME_COUNT):
        vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    success, img = vid.read()
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #toshow("Image Gray", imgGray)
    imgBlur = cv2.GaussianBlur(imgGray, (1,1), 2)
#     toshow("Image Blur", imgBlur)
#     imgCanny = cv2.Canny(imgBlur, 4)
#     toshow("Image Canny", imgCanny)
    imgThres = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 29, 18)
#     toshow("Image Threshold", imgThres)
    imgMed = cv2.medianBlur(imgThres,3)
#     toshow("Image Median", imgMed)
    CheckParking(imgMed)
    
    
    
    #for pos in posList:
    #    cv2.rectangle(img, pos, (pos[0]+width, pos[1]+height), (0,255,0), 2)
    
    
    toshow("Video", img)
    
    k=cv2.waitKey(1)
    if k==ord("q"):
        cv2.destroyAllWindows()
        break

#### Thank you!
#### Kuldeep Rajpoot